# 3. Preencher Dados de CPF e Dependencia

Este notebook identifica beneficiarios com CPF ou DEPENDENCIA vazios, busca os dados na base da Unimed e gera arquivos de apoio para preenchimento e validacao.

**Entradas necessarias:**
- Planilha de dados (`dados.xlsx`) na aba do mes de referencia
- Planilha de beneficiarios ativos da Unimed (`beneficiarios_ativos.xls`)

**Saidas:**
- Arquivo `beneficiarios_preenchimento.xlsx` (base para preenchimento)
- Arquivo `dados_preenchidos.xlsx` (dados com CPF/DEPENDENCIA preenchidos)
- Arquivo `nomes_nao_encontrados.xlsx` (familias nao encontradas na base da Unimed)

In [ ]:
import pandas as pd

caminho_dados = '../Data/'                  # Caminho para a planilha de dados           
caminho_beneficiarios_ativos = '../Data/'   # Caminho para a planilha de beneficiarios ativos  
mes = ''                                    # Mês Atual                                         


dados = pd.read_excel(caminho_dados, skiprows=1, sheet_name=mes)
ativos = pd.read_excel(caminho_beneficiarios_ativos)

In [ ]:
import unicodedata

caminho_preenchimento = '../Data/beneficiarios_preenchimento.xlsx'

colunas_esperadas = {'NOME', 'CPF', 'DEPENDENCIA'}
if not colunas_esperadas.issubset(dados.columns):
    raise ValueError(f"Planilha de dados sem as colunas esperadas: {colunas_esperadas}")
if not colunas_esperadas.issubset(ativos.columns):
    raise ValueError(f"Planilha da Unimed sem as colunas esperadas: {colunas_esperadas}")

filtro_pendentes = (
    dados['NOME'].notna()
    & (dados['NOME'].astype(str).str.strip() != '')
    & (dados['CPF'].isna() | dados['DEPENDENCIA'].isna())
)

pendentes = dados.loc[filtro_pendentes, ['NOME', 'CPF', 'DEPENDENCIA']].copy()
nomes_pendentes = pendentes['NOME'].astype(str).str.strip().drop_duplicates().tolist()

pendentes['NOME_CHAVE'] = pendentes['NOME'].astype(str).str.strip().str.upper()
ativos_aux = ativos.copy()
ativos_aux['NOME_CHAVE'] = ativos_aux['NOME'].astype(str).str.strip().str.upper()

resultado = (
    ativos_aux[ativos_aux['NOME_CHAVE'].isin(pendentes['NOME_CHAVE'])]
    [['NOME', 'DEPENDENCIA', 'CPF']]
    .drop_duplicates()
    .reset_index(drop=True)
)

resultado.to_excel(caminho_preenchimento, index=False)

print(f'Total de nomes pendentes encontrados nos dados: {len(nomes_pendentes)}')
print(f'Total de registros encontrados na Unimed: {len(resultado)}')
print(f'Arquivo gerado: {caminho_preenchimento}')

In [ ]:
caminho_saida_preenchido = '../Data/dados_preenchidos.xlsx'
caminho_saida_nao_encontrados = '../Data/nomes_nao_encontrados.xlsx'

dados_original = pd.read_excel(caminho_dados, skiprows=1, sheet_name=mes)
preenchimento = pd.read_excel(caminho_preenchimento)


def normalizar_nome_coluna(nome):
    nome = str(nome).strip().upper()
    nome = ''.join(
        c for c in unicodedata.normalize('NFKD', nome)
        if not unicodedata.combining(c)
    )
    return ' '.join(nome.split())


def mapear_colunas(df, colunas_requeridas):
    mapa_normalizado = {normalizar_nome_coluna(c): c for c in df.columns}
    mapeamento = {}
    faltando = []

    for col in colunas_requeridas:
        chave = normalizar_nome_coluna(col)
        if chave in mapa_normalizado:
            mapeamento[col] = mapa_normalizado[chave]
        else:
            faltando.append(col)

    if faltando:
        raise ValueError(f'Colunas ausentes na planilha: {faltando}')

    return mapeamento


colunas_dados_desejadas = ['CÓD DA FAMILIA', 'CARTEIRA', 'DEPENDENCIA', 'NOME', 'CPF']
mapa_dados = mapear_colunas(dados_original, colunas_dados_desejadas)
mapa_preenchimento = mapear_colunas(preenchimento, ['NOME', 'CPF', 'DEPENDENCIA'])

col_cod_familia = mapa_dados['CÓD DA FAMILIA']
col_carteira = mapa_dados['CARTEIRA']
col_dependencia = mapa_dados['DEPENDENCIA']
col_nome = mapa_dados['NOME']
col_cpf = mapa_dados['CPF']

col_nome_p = mapa_preenchimento['NOME']
col_cpf_p = mapa_preenchimento['CPF']
col_dependencia_p = mapa_preenchimento['DEPENDENCIA']

dados_preenchidos = dados_original.copy()
dados_preenchidos['NOME_CHAVE'] = dados_preenchidos[col_nome].astype(str).str.strip().str.upper()
preenchimento_aux = preenchimento.copy()
preenchimento_aux['NOME_CHAVE'] = preenchimento_aux[col_nome_p].astype(str).str.strip().str.upper()

preenchimento_por_nome = (
    preenchimento_aux.groupby('NOME_CHAVE', as_index=False)
    .agg({
        col_cpf_p: lambda s: s.dropna().iloc[0] if not s.dropna().empty else pd.NA,
        col_dependencia_p: lambda s: s.dropna().iloc[0] if not s.dropna().empty else pd.NA,
    })
    .rename(columns={
        col_cpf_p: 'CPF_NOVO',
        col_dependencia_p: 'DEPENDENCIA_NOVO'
    })
)

merge = dados_preenchidos.merge(
    preenchimento_por_nome,
    on='NOME_CHAVE',
    how='left'
)

cpf_vazio = merge[col_cpf].isna()
dependencia_vazia = merge[col_dependencia].isna()

merge.loc[cpf_vazio, col_cpf] = merge.loc[cpf_vazio, 'CPF_NOVO']
merge.loc[dependencia_vazia, col_dependencia] = merge.loc[dependencia_vazia, 'DEPENDENCIA_NOVO']

mask_com_pendencia = (
    merge[col_nome].notna()
    & (merge[col_nome].astype(str).str.strip() != '')
    & (dados_original[col_cpf].isna() | dados_original[col_dependencia].isna())
)

mask_nao_encontrado = (
    mask_com_pendencia
    & merge['CPF_NOVO'].isna()
    & merge['DEPENDENCIA_NOVO'].isna()
)

familias_nao_encontradas = merge.loc[mask_nao_encontrado, col_cod_familia].dropna().unique()

familia_completa = merge[merge[col_cod_familia].isin(familias_nao_encontradas)]
sem_familia = merge[mask_nao_encontrado & merge[col_cod_familia].isna()]

nao_encontrados_final = (
    pd.concat([familia_completa, sem_familia], ignore_index=True)
    [[col_cod_familia, col_carteira, col_dependencia, col_nome, col_cpf]]
    .drop_duplicates()
    .rename(columns={
        col_cod_familia: 'CÓD DA FAMILIA',
        col_carteira: 'CARTEIRA',
        col_dependencia: 'DEPENDENCIA',
        col_nome: 'NOME',
        col_cpf: 'CPF'
    })
    .sort_values(['CÓD DA FAMILIA', 'NOME'], na_position='last')
    .reset_index(drop=True)
)

colunas_saida = ['CÓD DA FAMILIA', 'CARTEIRA', 'DEPENDENCIA', 'NOME', 'CPF']
blocos_familia = []

for _, grupo in nao_encontrados_final.groupby('CÓD DA FAMILIA', dropna=False, sort=False):
    blocos_familia.append(grupo)
    blocos_familia.append(pd.DataFrame([{col: pd.NA for col in colunas_saida}]))

if blocos_familia:
    nao_encontrados_formatado = pd.concat(blocos_familia, ignore_index=True).iloc[:-1].reset_index(drop=True)
else:
    nao_encontrados_formatado = nao_encontrados_final.copy()

saida_final = merge.drop(columns=['NOME_CHAVE', 'CPF_NOVO', 'DEPENDENCIA_NOVO'])
saida_final.to_excel(caminho_saida_preenchido, index=False)
nao_encontrados_formatado.to_excel(caminho_saida_nao_encontrados, index=False)

print(f'Arquivo preenchido gerado: {caminho_saida_preenchido}')
print(f'Nomes nao encontrados gerado: {caminho_saida_nao_encontrados}')
print(f'Total de registros nao encontrados (familia completa): {len(nao_encontrados_final)}')